In [ ]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter

from nuplan.planning.utils.multithreading.worker_sequential import Sequential
# from nuplan.planning.utils.multithreading.worker_ray import RayDistributed




In [ ]:
import os, psutil


def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")


split = "nuplan_mini_val"

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
scene_tokens = None
scene_tokens = ["2283aea39bc1505e"]
log_names = None

scene_filter = SceneFilter(
    split_names=[split],
    log_names=log_names,
    scene_tokens=scene_tokens,
    duration_s=15.1,
    history_s=1.0,
)
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print("num scenes found", len(scenes))

In [ ]:
from asim.common.visualization.matplotlib.plots import render_scene_animation

scene = scenes[0]


render_scene_animation(scene, "", fps=10, end_idx=None, step=1, dpi=100, format="gif")